In [1]:
!pip install vosk
!pip install SpeechRecognition
!pip install pyaudio

     ---------------------------------------- 0.0/32.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/32.8 MB 1.4 MB/s eta 0:00:25
     --------------------------------------- 0.1/32.8 MB 751.6 kB/s eta 0:00:44
     --------------------------------------- 0.1/32.8 MB 595.3 kB/s eta 0:00:55
     --------------------------------------- 0.1/32.8 MB 655.8 kB/s eta 0:00:50
     --------------------------------------- 0.2/32.8 MB 811.5 kB/s eta 0:00:41
     --------------------------------------- 0.3/32.8 MB 806.1 kB/s eta 0:00:41
     --------------------------------------- 0.3/32.8 MB 840.2 kB/s eta 0:00:39
      -------------------------------------- 0.5/32.8 MB 994.6 kB/s eta 0:00:33
      -------------------------------------- 0.5/32.8 MB 962.6 kB/s eta 0:00:34
      --------------------------------------- 0.5/32.8 MB 1.0 MB/s eta 0:00:33
      --------------------------------------- 0.6/32.8 MB 1.0 MB/s eta 0:00:32
      -------------------------------------- 0.7/32

In [5]:
from vosk import Model, KaldiRecognizer  # оффлайн-распознавание от Vosk
import speech_recognition  # распознавание пользовательской речи (Speech-To-Text)
import wave  # создание и чтение аудиофайлов формата wav
import json  # работа с json-файлами и json-строками
import os  # работа с файловой системой

modelpath = "/home/student/Документы/vosk/vosk-model-small-ru-0.22"

def record_and_recognize_audio(*args: tuple):
    """
    Запись и распознавание аудио
    """
    with microphone:
        recognized_data = ""

        # регулирование уровня окружающего шума
        recognizer.adjust_for_ambient_noise(microphone, duration=2)

        try:
            print("Listening...")
            audio = recognizer.listen(microphone, 5, 5)

            with open("microphone-results.wav", "wb") as file:
                file.write(audio.get_wav_data())

        except speech_recognition.WaitTimeoutError:
            print("Can you check if your microphone is on, please?")
            return

        # использование online-распознавания через Google
        try:
            print("Started recognition...")
            recognized_data = recognizer.recognize_google(audio, language="ru").lower()

        except speech_recognition.UnknownValueError:
            pass

        # в случае проблем с доступом в Интернет происходит попытка
        # использовать offline-распознавание через Vosk
        except speech_recognition.RequestError:
            print("Trying to use offline recognition...")
            recognized_data = use_offline_recognition()

        return recognized_data


def use_offline_recognition():
    """
    Переключение на оффлайн-распознавание речи
    :return: распознанная фраза
    """
    recognized_data = ""
    try:
        # проверка наличия модели на нужном языке в каталоге приложения
        if not os.path.exists(modelpath):
            print(
                "Please download the model from:\n"
                "https://alphacephei.com/vosk/models and unpack as 'model' in the current folder."
            )
            exit(1)

        # анализ записанного в микрофон аудио (чтобы избежать повторов фразы)
        wave_audio_file = wave.open("microphone-results.wav", "rb")
        model = Model(modelpath)
        offline_recognizer = KaldiRecognizer(model, wave_audio_file.getframerate())

        data = wave_audio_file.readframes(wave_audio_file.getnframes())
        if len(data) > 0:
            if offline_recognizer.AcceptWaveform(data):
                recognized_data = offline_recognizer.Result()

                # получение данных распознанного текста из JSON-строки
                # (чтобы можно было выдать по ней ответ)
                recognized_data = json.loads(recognized_data)
                recognized_data = recognized_data["text"]
    except:
        print("Sorry, speech service is unavailable. Try again later")

    return recognized_data


if __name__ == "__main__":
    # инициализация инструментов распознавания и ввода речи
    recognizer = speech_recognition.Recognizer()
    microphone = speech_recognition.Microphone()

    while True:
        # старт записи речи с последующим выводом распознанной речи
        # и удалением записанного в микрофон аудио
        voice_input = record_and_recognize_audio()
        os.remove("microphone-results.wav")
        print(voice_input)

Listening...
Started recognition...


KeyboardInterrupt: 